In [1]:
# Code by Andrew England - 21285542 
import numpy as np
import pylab as plt
from sklearn.linear_model import BayesianRidge, LinearRegression
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
%matplotlib inline

In [3]:
#########import
X_train = np.loadtxt('input/X_train.csv', delimiter=',', skiprows=1)
X_test = np.loadtxt('input/X_test.csv', delimiter=',', skiprows=1)
y_train = np.loadtxt('input/y_train.csv', delimiter=',', skiprows=1)[:,1]

In [3]:
#########subset features - training
X = np.zeros((X_train.shape[0], 4))
X[:,0] = X_train[:,0]
X[:,1] = (X_train[:,1]+X_train[:,2])/2.0
X[:,2] = X_train[:,3] 
X[:,3] = (X_train[:,4]+X_train[:,5])/2.0

In [4]:
X_train = X

In [5]:
#######subset features - test
X = np.zeros((X_test.shape[0], 4))
X[:,0] = X_test[:,0]
X[:,1] = (X_test[:,1]+X_test[:,2])/2.0
X[:,2] = X_test[:,3] 
X[:,3] = (X_test[:,4]+X_test[:,5])/2.0

In [6]:
X_test = X

In [7]:
#X_train_temp, X_test_temp, y_train_temp, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=2018)


In [8]:
######################linear comparison###########################
linreg = LinearRegression()
linreg.fit(X_train, y_train)
ols_pred = linreg.predict(X_test)
ols_pred
#mean_absolute_error(y_test, ols_pred)

array([ 68.16398633,  87.39969359,  -6.16061846,  13.29067979,
       206.62251206, 259.6349414 ,  98.78959224, 163.00990684,
       207.79134113, 171.28701269,  16.34524149, 174.85084571,
        30.70948479, -11.62943219,  57.33642243, 217.17605882,
        -7.09619297, -17.43888425,  28.68908634, 206.16404466,
        51.77204932,  80.41285853,  56.19728755, -13.252061  ,
        22.67008888,  51.77204932,  -1.36060201, 104.82261941,
         4.21079328, 175.48123839,  14.77072521, -27.66826053,
       -17.43888425,   5.22826555, -11.69741721, 406.12728248,
       239.9912633 ,  37.26106977, -15.33486054, 461.4732589 ,
       332.75053386])

In [9]:
#####################polynomial########################
#########poly order = 2
#########compute important features in 1st order
Bay = BayesianRidge(compute_score=True)
Bay.fit(X_train, y_train)
Important_train = np.zeros((X_train.shape[0], X_train.shape[1]))
Important_test = np.zeros((X_test.shape[0], X_test.shape[1]))
j=0
for i in range(1, X_train.shape[1]):
    if Bay.sigma_[i-1,i-1]>1e-07:
        Important_train[:,j] = X_train[:,i-1]
        Important_test[:,j] = X_test[:,i-1]
        j = j+1
Xtrain_feats = Important_train[:, :j]
Xtest_feats = Important_test[:,:j]


In [10]:
Xtrain_feats.shape

(168, 3)

In [11]:
########poly order = 2
squaretrain = np.power(X_train,2)
squaretest = np.power(X_test,2)

In [12]:
#######compute important features in 2nd order
Bay_squared_feats = BayesianRidge()
Bay_squared_feats.fit(squaretrain, y_train)

Importantsquare_train = np.zeros((squaretrain.shape[0], squaretrain.shape[1]))
Importantsquare_test = np.zeros((squaretest.shape[0],squaretest.shape[1]))
k=0
for i in range(1, squaretrain.shape[1]):
    if Bay_squared_feats.sigma_[i-1,i-1]>1e-07:
        Importantsquare_train[:,k] = squaretrain[:,i-1]
        Importantsquare_test[:,k] = squaretest[:,i-1]
        k = k+1
Xtrain_squarefeats = Importantsquare_train[:, :k]
Xtest_squarefeats = Importantsquare_test[:,:k]

In [13]:
Xtrain_squarefeats.shape

(168, 1)

In [14]:
#######poly order = 3
threetrain = np.power(X_train,3)
threetest = np.power(X_test,3)

In [15]:
#######compute important features in 3nd order
Bay_three_feats = BayesianRidge()
Bay_three_feats.fit(threetrain, y_train)

Importantthree_train = np.zeros((threetrain.shape[0], threetrain.shape[1]))
Importantthree_test = np.zeros((threetest.shape[0],threetest.shape[1]))
l=0
for i in range(1, threetrain.shape[1]):
    if Bay_three_feats.sigma_[i-1,i-1]>1e-07:
        Importantthree_train[:,l] = threetrain[:,i-1]
        Importantthree_test[:,l] = threetest[:,i-1]
        l = l+1
Xtrain_threefeats = Importantthree_train[:, :l]
Xtest_threefeats = Importantthree_test[:,:l]

In [16]:
Xtrain_threefeats.shape

(168, 0)

In [17]:
######ceate design matrix
onestrain = np.ones((Xtrain_feats.shape[0],1))
onestest = np.ones((Xtest_feats.shape[0],1))

designtrain = np.concatenate((onestrain,Xtrain_feats, Xtrain_squarefeats, Xtrain_threefeats),axis=1)
designtest = np.concatenate((onestest,Xtest_feats, Xtest_squarefeats, Xtest_threefeats),axis=1)

In [18]:
designtrain.shape

(168, 5)

In [19]:
#########perform ridge regression
Bay_poly = BayesianRidge()
Bay_poly.fit(designtrain, y_train)
y_predicted = Bay_poly.predict(designtest)
y_predicted[y_predicted < 0] = 0
#print('3rd order Model MAE:', mean_absolute_error(y_test, y_predicted))
#print('Mean  MAE:', mean_absolute_error(y_test, np.full(y_test.shape, np.mean(y_test))))
    

In [20]:
#Bay_poly = BayesianRidge()
#Bay_poly.fit(designtrain, y_train)
#y_predicted = Bay_poly.predict(designtest)

In [21]:

df_sub = pd.DataFrame({'Id': np.arange(y_predicted.size), 'PRP': y_predicted})
df_sub.to_csv('21285542.csv', index=False)

In [22]:
print(y_predicted)

[7.92518093e+01 1.06182501e+02 0.00000000e+00 0.00000000e+00
 2.04105027e+02 2.78144258e+02 8.19171033e+01 1.57893203e+02
 1.94921471e+02 1.88371737e+02 1.36483876e+01 1.83475179e+02
 2.37201350e+01 0.00000000e+00 5.51265105e+01 2.20971386e+02
 1.68599412e+00 0.00000000e+00 5.00624880e+01 2.03988043e+02
 4.45650641e+01 5.44186252e+01 7.35522367e+01 3.86436273e-01
 4.07578959e+01 4.45650641e+01 0.00000000e+00 1.09477135e+02
 9.69383006e+00 1.83636033e+02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.20758056e+01 0.00000000e+00 4.17749119e+02
 2.57868047e+02 3.29522375e+01 0.00000000e+00 3.36671263e+02
 3.43443990e+02]
